## ایمبیڈنگز

پچھلی مثال میں، ہم نے `vocab_size` کی لمبائی والے ہائی ڈائمینشنل بیگ آف ورڈز ویکٹرز پر کام کیا، اور کم ڈائمینشنل پوزیشنل ریپریزنٹیشن ویکٹرز کو واضح طور پر اسپارس ون-ہاٹ ریپریزنٹیشن میں تبدیل کیا۔ یہ ون-ہاٹ ریپریزنٹیشن میموری کے لحاظ سے مؤثر نہیں ہے۔ اس کے علاوہ، ہر لفظ کو دوسرے الفاظ سے آزاد سمجھا جاتا ہے، اس لیے ون-ہاٹ انکوڈڈ ویکٹرز الفاظ کے درمیان معنوی مشابہت کو ظاہر نہیں کرتے۔

اس یونٹ میں، ہم **نیوز اے جی** ڈیٹاسیٹ کو مزید دریافت کریں گے۔ شروع کرنے کے لیے، آئیے ڈیٹا لوڈ کرتے ہیں اور پچھلے یونٹ سے کچھ تعریفیں حاصل کرتے ہیں۔


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### ایمبیڈنگ کیا ہے؟

**ایمبیڈنگ** کا تصور یہ ہے کہ الفاظ کو کم جہتی گھنے ویکٹرز کے ذریعے ظاہر کیا جائے جو لفظ کے معنوی مفہوم کو ظاہر کرتے ہیں۔ ہم بعد میں یہ بات کریں گے کہ بامعنی لفظی ایمبیڈنگز کیسے بنائی جائیں، لیکن فی الحال ایمبیڈنگز کو ایک ایسے طریقے کے طور پر سوچیں جو لفظی ویکٹر کی جہت کو کم کرتا ہے۔

ایمبیڈنگ لیئر ایک لفظ کو ان پٹ کے طور پر لیتی ہے اور ایک مخصوص `embedding_size` کے آؤٹ پٹ ویکٹر کو پیدا کرتی ہے۔ ایک لحاظ سے، یہ `Dense` لیئر سے بہت مشابہ ہے، لیکن یہ ایک-ہاٹ انکوڈڈ ویکٹر لینے کے بجائے، ایک لفظی نمبر کو ان پٹ کے طور پر لے سکتی ہے۔

جب ہم اپنے نیٹ ورک میں پہلی لیئر کے طور پر ایمبیڈنگ لیئر کا استعمال کرتے ہیں، تو ہم بیگ-آف-ورڈز ماڈل سے **ایمبیڈنگ بیگ** ماڈل میں منتقل ہو سکتے ہیں، جہاں ہم پہلے اپنے متن کے ہر لفظ کو اس کے متعلقہ ایمبیڈنگ میں تبدیل کرتے ہیں، اور پھر ان تمام ایمبیڈنگز پر کوئی مجموعی فنکشن جیسے کہ `sum`، `average` یا `max` کا حساب لگاتے ہیں۔

![پانچ سیکوئنس الفاظ کے لیے ایمبیڈنگ کلاسیفائر کی تصویر۔](../../../../../translated_images/ur/embedding-classifier-example.b77f021a7ee67eee.webp)

ہمارے کلاسیفائر نیورل نیٹ ورک میں درج ذیل لیئرز شامل ہیں:

* `TextVectorization` لیئر، جو ایک سٹرنگ کو ان پٹ کے طور پر لیتی ہے اور ٹوکن نمبرز کا ایک ٹینسر پیدا کرتی ہے۔ ہم ایک معقول `vocab_size` مقرر کریں گے اور کم استعمال ہونے والے الفاظ کو نظرانداز کریں گے۔ ان پٹ کی شکل 1 ہوگی، اور آؤٹ پٹ کی شکل $n$ ہوگی، کیونکہ نتیجے میں ہمیں $n$ ٹوکنز ملیں گے، جن میں سے ہر ایک میں 0 سے `vocab_size` تک کے نمبر ہوں گے۔
* `Embedding` لیئر، جو $n$ نمبرز لیتی ہے اور ہر نمبر کو ایک دی گئی لمبائی (ہمارے مثال میں 100) کے گھنے ویکٹر میں کم کرتی ہے۔ اس طرح، $n$ کی شکل کا ان پٹ ٹینسر $n\times 100$ کی شکل کے ٹینسر میں تبدیل ہو جائے گا۔
* ایگریگیشن لیئر، جو اس ٹینسر کا پہلے محور کے ساتھ اوسط نکالتی ہے، یعنی یہ مختلف الفاظ کے مطابق تمام $n$ ان پٹ ٹینسرز کا اوسط نکالے گی۔ اس لیئر کو نافذ کرنے کے لیے، ہم ایک `Lambda` لیئر استعمال کریں گے اور اس میں اوسط نکالنے کا فنکشن پاس کریں گے۔ آؤٹ پٹ کی شکل 100 ہوگی، اور یہ پورے ان پٹ سیکوئنس کی عددی نمائندگی ہوگی۔
* آخری `Dense` لکیری کلاسیفائر۔


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` کے پرنٹ آؤٹ میں، **output shape** کالم میں پہلے ٹینسر ڈائمینشن `None` مائنی بیچ کے سائز کو ظاہر کرتا ہے، اور دوسرا ٹوکن سیکوئنس کی لمبائی کو ظاہر کرتا ہے۔ مائنی بیچ میں موجود تمام ٹوکن سیکوئنسز کی لمبائیاں مختلف ہوتی ہیں۔ ہم اگلے سیکشن میں اس سے نمٹنے کے طریقے پر بات کریں گے۔

اب نیٹ ورک کو ٹرین کرتے ہیں:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


**نوٹ** کہ ہم ڈیٹا کے ایک ذیلی حصے کی بنیاد پر ویکٹرائزر بنا رہے ہیں۔ یہ عمل کو تیز کرنے کے لیے کیا جاتا ہے، اور اس کے نتیجے میں ایسی صورتحال پیدا ہو سکتی ہے جب ہمارے متن کے تمام ٹوکنز لغت میں موجود نہ ہوں۔ اس صورت میں، ان ٹوکنز کو نظر انداز کر دیا جائے گا، جو تھوڑی کم درستگی کا باعث بن سکتا ہے۔ تاہم، حقیقی زندگی میں متن کا ایک ذیلی حصہ اکثر لغت کا اچھا تخمینہ فراہم کرتا ہے۔


### متغیر ترتیب کے سائز سے نمٹنا

آئیے سمجھتے ہیں کہ منی بیچز میں تربیت کیسے ہوتی ہے۔ اوپر دیے گئے مثال میں، ان پٹ ٹینسر کا طول 1 ہے، اور ہم 128-لمبے منی بیچز استعمال کرتے ہیں، اس طرح ٹینسر کا اصل سائز $128 \times 1$ ہوتا ہے۔ تاہم، ہر جملے میں ٹوکنز کی تعداد مختلف ہوتی ہے۔ اگر ہم `TextVectorization` لیئر کو ایک انفرادی ان پٹ پر لاگو کریں، تو واپس کیے گئے ٹوکنز کی تعداد مختلف ہوگی، اس بات پر منحصر ہے کہ متن کو کیسے ٹوکنائز کیا گیا ہے:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


تاہم، جب ہم ویکٹرائزر کو کئی سلسلوں پر لاگو کرتے ہیں، تو اسے مستطیل شکل کا ٹینسر پیدا کرنا ہوتا ہے، اس لیے یہ غیر استعمال شدہ عناصر کو پیڈ ٹوکن (جو ہمارے معاملے میں صفر ہے) سے بھر دیتا ہے۔


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

یہاں ہم ایمبیڈنگز دیکھ سکتے ہیں:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **نوٹ**: پیڈنگ کی مقدار کو کم کرنے کے لیے، بعض صورتوں میں یہ مناسب ہوتا ہے کہ ڈیٹا سیٹ میں تمام سلسلوں کو بڑھتی ہوئی لمبائی (یا زیادہ درست طور پر، ٹوکنز کی تعداد) کے ترتیب میں رکھا جائے۔ یہ یقینی بنائے گا کہ ہر منی بیچ میں ایک جیسی لمبائی کے سلسلے شامل ہوں۔


## سیمینٹک ایمبیڈنگز: ورڈ2ویک

پچھلی مثال میں، ایمبیڈنگ لیئر نے الفاظ کو ویکٹر ریپریزنٹیشنز میں میپ کرنا سیکھا، لیکن ان ریپریزنٹیشنز میں سیمینٹک معنی نہیں تھے۔ یہ بہتر ہوگا کہ ایک ایسا ویکٹر ریپریزنٹیشن سیکھا جائے جس میں مشابہ الفاظ یا مترادفات ایسے ویکٹرز کے قریب ہوں جو کسی ویکٹر فاصلے (مثلاً یُوکلیڈین فاصلے) کے لحاظ سے قریب ہوں۔

اس مقصد کے لیے، ہمیں اپنے ایمبیڈنگ ماڈل کو ایک بڑی ٹیکسٹ کلیکشن پر پہلے سے تربیت دینا ہوگا، جیسے کہ [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) تکنیک کا استعمال کرتے ہوئے۔ یہ دو بنیادی آرکیٹیکچرز پر مبنی ہے جو الفاظ کی ڈسٹریبیوٹڈ ریپریزنٹیشن پیدا کرنے کے لیے استعمال ہوتے ہیں:

 - **کنٹینیوس بیگ آف ورڈز** (CBoW)، جہاں ماڈل کو ارد گرد کے کانٹیکسٹ سے ایک لفظ کی پیش گوئی کرنے کے لیے تربیت دی جاتی ہے۔ دیے گئے این گرام $(W_{-2},W_{-1},W_0,W_1,W_2)$ میں، ماڈل کا مقصد $W_0$ کو $(W_{-2},W_{-1},W_1,W_2)$ سے پیش گوئی کرنا ہے۔
 - **کنٹینیوس اسکیپ-گرام** CBoW کے برعکس ہے۔ ماڈل موجودہ لفظ کی پیش گوئی کے لیے ارد گرد کے کانٹیکسٹ الفاظ کی ونڈو استعمال کرتا ہے۔

CBoW تیز ہے، جبکہ اسکیپ-گرام سست ہے، لیکن یہ کم استعمال ہونے والے الفاظ کی بہتر نمائندگی کرتا ہے۔

![تصویر جو CBoW اور اسکیپ-گرام الگورتھمز کو الفاظ کو ویکٹرز میں تبدیل کرنے کے لیے دکھا رہی ہے۔](../../../../../translated_images/ur/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

گوگل نیوز ڈیٹاسیٹ پر پری ٹرینڈ Word2Vec ایمبیڈنگ کے ساتھ تجربہ کرنے کے لیے، ہم **gensim** لائبریری استعمال کر سکتے ہیں۔ نیچے ہم 'neural' کے سب سے مشابہ الفاظ تلاش کرتے ہیں۔

> **نوٹ:** جب آپ پہلی بار ویکٹرز بناتے ہیں، تو انہیں ڈاؤنلوڈ کرنے میں کچھ وقت لگ سکتا ہے!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ہم لفظ سے ویکٹر ایمبیڈنگ بھی نکال سکتے ہیں، جسے درجہ بندی ماڈل کی تربیت میں استعمال کیا جا سکتا ہے۔ ایمبیڈنگ میں 300 اجزاء ہوتے ہیں، لیکن یہاں وضاحت کے لیے ہم صرف ویکٹر کے پہلے 20 اجزاء دکھاتے ہیں:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

زبردست بات سیمینٹک ایمبیڈنگز کے بارے میں یہ ہے کہ آپ سیمینٹکس کی بنیاد پر ویکٹر انکوڈنگ کو تبدیل کر سکتے ہیں۔ مثال کے طور پر، ہم ایک ایسا لفظ تلاش کرنے کے لئے کہہ سکتے ہیں جس کی ویکٹر نمائندگی *king* اور *woman* کے الفاظ کے جتنا قریب ہو سکے، اور *man* کے لفظ سے جتنا دور ہو سکے:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ایک مثال اوپر کچھ اندرونی GenSym جادو استعمال کرتی ہے، لیکن بنیادی منطق حقیقت میں کافی سادہ ہے۔ ایمبیڈنگز کے بارے میں ایک دلچسپ بات یہ ہے کہ آپ ایمبیڈنگ ویکٹرز پر عام ویکٹر آپریشنز انجام دے سکتے ہیں، اور یہ آپریشنز الفاظ کے **معانی** پر اثر انداز ہوں گے۔ اوپر دی گئی مثال کو ویکٹر آپریشنز کی شرائط میں بیان کیا جا سکتا ہے: ہم **KING-MAN+WOMAN** کے مطابق ویکٹر کا حساب لگاتے ہیں (آپریشنز `+` اور `-` متعلقہ الفاظ کے ویکٹر نمائندگیوں پر انجام دیے جاتے ہیں)، اور پھر اس ویکٹر کے قریب ترین لفظ کو لغت میں تلاش کرتے ہیں۔


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **نوٹ**: ہمیں *مرد* اور *عورت* کے ویکٹرز میں ایک چھوٹا سا عدد شامل کرنا پڑا - اسے ہٹا کر دیکھیں کہ کیا ہوتا ہے۔

قریب ترین ویکٹر تلاش کرنے کے لیے، ہم TensorFlow کے آلات استعمال کرتے ہیں تاکہ ہمارے ویکٹر اور لغت میں موجود تمام ویکٹرز کے درمیان فاصلوں کا ایک ویکٹر حساب کریں، اور پھر `argmin` کا استعمال کرتے ہوئے کم سے کم لفظ کا انڈیکس تلاش کریں۔


جبکہ Word2Vec الفاظ کے معنی کو ظاہر کرنے کا ایک بہترین طریقہ لگتا ہے، اس کے کئی نقصانات ہیں، جن میں شامل ہیں:

* CBoW اور skip-gram ماڈلز دونوں **predictive embeddings** ہیں، اور یہ صرف مقامی سیاق و سباق کو مدنظر رکھتے ہیں۔ Word2Vec عالمی سیاق و سباق کا فائدہ نہیں اٹھاتا۔
* Word2Vec الفاظ کی **morphology** کو مدنظر نہیں رکھتا، یعنی یہ حقیقت کہ کسی لفظ کا مطلب اس کے مختلف حصوں، جیسے جڑ، پر منحصر ہو سکتا ہے۔

**FastText** دوسرے نقصان کو دور کرنے کی کوشش کرتا ہے اور Word2Vec پر مبنی ہے، ہر لفظ اور اس کے اندر موجود کردار n-grams کے لیے ویکٹر نمائندگی سیکھ کر۔ ان نمائندگیوں کی قدریں ہر تربیتی مرحلے پر ایک ویکٹر میں اوسط کی جاتی ہیں۔ اگرچہ یہ پری ٹریننگ میں اضافی حساب کتاب شامل کرتا ہے، یہ لفظ embeddings کو ذیلی لفظ معلومات کو انکوڈ کرنے کے قابل بناتا ہے۔

ایک اور طریقہ، **GloVe**، لفظ embeddings کے لیے ایک مختلف نقطہ نظر استعمال کرتا ہے، جو لفظ-سیاق و سباق میٹرکس کی factorization پر مبنی ہے۔ پہلے، یہ ایک بڑا میٹرکس بناتا ہے جو مختلف سیاق و سباق میں لفظ کے وقوعات کی تعداد شمار کرتا ہے، اور پھر یہ کوشش کرتا ہے کہ اس میٹرکس کو کم جہتوں میں اس طرح ظاہر کرے کہ reconstruction loss کم سے کم ہو۔

gensim لائبریری ان لفظ embeddings کی حمایت کرتی ہے، اور آپ اوپر دیے گئے ماڈل لوڈنگ کوڈ کو تبدیل کرکے ان کے ساتھ تجربہ کر سکتے ہیں۔


## کیراس میں پہلے سے تربیت یافتہ ایمبیڈنگز کا استعمال

ہم اوپر دیے گئے مثال کو اس طرح تبدیل کر سکتے ہیں کہ اپنی ایمبیڈنگ لیئر کے میٹرکس کو معنوی ایمبیڈنگز، جیسے کہ Word2Vec، سے پہلے سے بھر دیں۔ پہلے سے تربیت یافتہ ایمبیڈنگ اور ٹیکسٹ کارپس کے ذخیرہ الفاظ ممکنہ طور پر ایک جیسے نہیں ہوں گے، اس لیے ہمیں ایک کا انتخاب کرنا ہوگا۔ یہاں ہم دو ممکنہ اختیارات کا جائزہ لیتے ہیں: ٹوکنائزر کے ذخیرہ الفاظ کا استعمال، اور Word2Vec ایمبیڈنگز کے ذخیرہ الفاظ کا استعمال۔

### ٹوکنائزر کے ذخیرہ الفاظ کا استعمال

جب ٹوکنائزر کے ذخیرہ الفاظ کا استعمال کیا جاتا ہے، تو ذخیرہ الفاظ کے کچھ الفاظ کے لیے Word2Vec ایمبیڈنگز موجود ہوں گی، اور کچھ غائب ہوں گے۔ فرض کریں کہ ہمارے ذخیرہ الفاظ کا سائز `vocab_size` ہے، اور Word2Vec ایمبیڈنگ ویکٹر کی لمبائی `embed_size` ہے، تو ایمبیڈنگ لیئر ایک وزن میٹرکس کی شکل میں ظاہر ہوگی جس کا سائز `vocab_size`$\times$`embed_size` ہوگا۔ ہم اس میٹرکس کو ذخیرہ الفاظ کے ذریعے بھر کر مکمل کریں گے:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


ورڈ2ویک کے ذخیرہ الفاظ میں موجود نہ ہونے والے الفاظ کے لیے، ہم یا تو انہیں صفر کے طور پر چھوڑ سکتے ہیں، یا ایک تصادفی ویکٹر بنا سکتے ہیں۔

اب ہم پری ٹرینڈ وزن کے ساتھ ایک ایمبیڈنگ لیئر تعریف کر سکتے ہیں:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **نوٹ**: دھیان دیں کہ ہم نے `Embedding` بناتے وقت `trainable=False` سیٹ کیا ہے، جس کا مطلب ہے کہ ہم Embedding لیئر کو دوبارہ تربیت نہیں دے رہے ہیں۔ اس کی وجہ سے درستگی تھوڑی کم ہو سکتی ہے، لیکن یہ تربیت کے عمل کو تیز کر دیتا ہے۔

### ایمبیڈنگ کے ذخیرہ الفاظ کا استعمال

پچھلے طریقے کے ساتھ ایک مسئلہ یہ ہے کہ TextVectorization اور Embedding میں استعمال ہونے والے ذخیرہ الفاظ مختلف ہیں۔ اس مسئلے کو حل کرنے کے لیے، ہم درج ذیل میں سے کوئی ایک حل استعمال کر سکتے ہیں:
* Word2Vec ماڈل کو ہمارے ذخیرہ الفاظ پر دوبارہ تربیت دیں۔
* ہمارے ڈیٹا سیٹ کو پہلے سے تربیت یافتہ Word2Vec ماڈل کے ذخیرہ الفاظ کے ساتھ لوڈ کریں۔ ڈیٹا سیٹ کو لوڈ کرتے وقت استعمال ہونے والے ذخیرہ الفاظ کو لوڈنگ کے دوران مخصوص کیا جا سکتا ہے۔

دوسرا طریقہ زیادہ آسان لگتا ہے، تو آئیے اسے نافذ کرتے ہیں۔ سب سے پہلے، ہم Word2Vec ایمبیڈنگ سے لیے گئے مخصوص ذخیرہ الفاظ کے ساتھ ایک `TextVectorization` لیئر بنائیں گے:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

لائبریری gensim لفظ embeddings میں ایک آسان فنکشن `get_keras_embeddings` شامل ہے، جو خود بخود آپ کے لئے متعلقہ Keras embeddings layer تخلیق کرے گا۔


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


ہماری ڈیٹاسیٹ کے کچھ الفاظ پری ٹرینڈ گلوو وکیبلری میں موجود نہیں ہیں، اور اسی وجہ سے انہیں بنیادی طور پر نظرانداز کیا جاتا ہے۔ اس مسئلے کو حل کرنے کے لیے، ہم اپنی ڈیٹاسیٹ کی بنیاد پر اپنی ایمبیڈنگز کو ٹرین کر سکتے ہیں۔


## سیاقی ایمبیڈنگز

روایتی پری ٹرینڈ ایمبیڈنگز جیسے Word2Vec کی ایک اہم حد یہ ہے کہ، اگرچہ وہ کسی لفظ کے کچھ معنی کو سمجھ سکتے ہیں، وہ مختلف معنوں کے درمیان فرق نہیں کر سکتے۔ یہ نیچے والے ماڈلز میں مسائل پیدا کر سکتا ہے۔

مثال کے طور پر، لفظ 'play' کے مختلف معنی ہیں ان دو جملوں میں:
- میں تھیٹر میں ایک **play** دیکھنے گیا۔
- جان اپنے دوستوں کے ساتھ **play** کرنا چاہتا ہے۔

ہم نے جن پری ٹرینڈ ایمبیڈنگز کی بات کی وہ لفظ 'play' کے دونوں معنوں کو ایک ہی ایمبیڈنگ میں ظاہر کرتی ہیں۔ اس حد کو دور کرنے کے لیے، ہمیں **زبان ماڈل** کی بنیاد پر ایمبیڈنگز بنانی ہوں گی، جو ایک بڑے متن کے مجموعے پر تربیت یافتہ ہوتا ہے اور *جانتا ہے* کہ الفاظ کو مختلف سیاق و سباق میں کیسے جوڑا جا سکتا ہے۔ سیاقی ایمبیڈنگز پر تفصیل سے بات کرنا اس سبق کے دائرہ کار سے باہر ہے، لیکن ہم ان پر واپس آئیں گے جب اگلے یونٹ میں زبان ماڈلز پر بات کریں گے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
